# LLM-based Initial Filtering

Since we couldn't easily filter stuff like minimum education requirements and experience, let's try using an LLM to filter for that

In [12]:
import json
from pydantic import BaseModel, Field
from pprint import pprint

from utils.with_structured_output import with_structured_output

In [3]:
with open("../output/parsed_resume_info.json", "r") as file:
    resume_info = json.load(file)

with open("../output/parsed_job_desc.json", "r") as file:
    job_info = json.load(file)

## Filter Education

In [20]:
class MinReq(BaseModel):
    meets_req: bool = Field(..., alias="Meets Requirements")
    reasoning: str = Field(..., alias="Reasoning")

In [15]:
FILTER_EDUCATION_TEMPLATE = """
You are an expert resume evaluator. Given a resume's education section and a job description's minimum education requirements, your job is to determine whether the resume meets the minimum requirements.

Both the resume and job description education information will be provided as JSON strings.

Format your output as follows:
    {{
        "Meets Requirements": <Boolean indicating whether the resume meets the minimum education requirements>
        "Reasoning": "Brief reasoning (1-2 sentences) behind the decision."
    }}
    
Resume's Education Information:
{resume_edu}

Job Description Minimum Education Requirements:
{min_education_req}

Output:
"""

In [16]:
meets_req = with_structured_output(
    prompt=FILTER_EDUCATION_TEMPLATE.format(
        resume_edu=resume_info["Education"],
        min_education_req=job_info["Required Education"]),
    schema=MinReq)

In [17]:
pprint(meets_req)

{'Meets Requirements': True,
 'Reasoning': 'The resume lists a degree from Texas A&M University with a '
              "major in Computer Science, which matches the job description's "
              'minimum education requirements.'}


## Filter Experience

In [21]:
FILTER_EXPERIENCE_TEMPLATE = """
You are an expert resume evaluator. Given a resume and job description's minimum experience requirements, your job is to determine whether the resume meets the **minimum** experience requirements.

Both the resume and job description are represented as JSON strings.

Format your output as follows:
    {{
        "Meets Requirements": <Boolean indicating whether the resume meets the minimum experience requirements>
        "Reasoning": "Brief reasoning (1-2 sentences) behind the decision."
    }}
    
Resume YOE: {resume_yoe} years

Resume Skill Information:
{resume_skills}

Job Description Minimum Experience Requirements:
{min_experience_req}

Output:
"""

In [26]:
meets_req = with_structured_output(
    prompt=FILTER_EXPERIENCE_TEMPLATE.format(
        resume_yoe=resume_info["Work Experience"]["Total Years of Experience"]
        resume_skills=resume_info["Skills"]["Technical Skills"] + resume_info["Skills"]["Domain-Specific Skills"],
        min_experience_req=job_info["Required Experience"]),
    schema=MinReq)

In [27]:
pprint(meets_req)

{'Meets Requirements': False,
 'Reasoning': 'The resume does not mention any direct experience with Linux '
              'kernel or application-level system software, which are '
              "explicitly mentioned in the job description's minimum "
              'experience requirements.'}
